In [4]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os 
import json
from pathlib import Path

#define groq key 
groq_key = "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" # os.environ['groqkey']

In [5]:
#model variables
llama_31 = 'llama-3.1-70b-versatile'
llama_3_1 = ChatGroq(
    temperature=0,
    model=llama_31,
    api_key=groq_key 
)

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
base_embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
text_splitter = RecursiveCharacterTextSplitter()

/var/folders/7y/69fhwyms35b2vywqpl4sxt700000gn/T/ipykernel_1175/1026861699.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  base_embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/opt/anaconda3/envs/capstone/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [7]:
base_path = Path("/Users/yarg/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/DataSci210_MedicationDeprescriber/Data")

noteConcepts = pd.read_csv(base_path / 'noteConcepts.txt', sep="|")
encounters = pd.read_csv(base_path / 'encounters.txt', sep="|")
orders = pd.read_csv(base_path / 'orders.txt', sep="|")
hospitalAcquiredDx = pd.read_csv(base_path / 'hospitalAcquiredDx.txt', sep="|")
noteText = pd.read_csv(base_path / 'noteText.txt', sep="|")
presentOnAdmitDx = pd.read_csv(base_path / 'presentOnAdmitDx.txt', sep="|")
#select random EncounterKey
encounterKey = encounters.sample(1)['EncounterKey'].values[0]

In [8]:
encounterKey

'D8FCAB3DB25660'

In [4]:
#filter all dfs to the selected EncounterKey
noteConcepts = noteConcepts[noteConcepts['EncounterKey'] == encounterKey]
orders = orders[orders['EncounterKey'] == encounterKey]
hospitalAcquiredDx = hospitalAcquiredDx[hospitalAcquiredDx['EncounterKey'] == encounterKey]
noteText = noteText[noteText['EncounterKey'] == encounterKey]
presentOnAdmitDx = presentOnAdmitDx[presentOnAdmitDx['EncounterKey'] == encounterKey]
encounters = encounters[encounters['EncounterKey'] == encounterKey]

In [5]:
hospitalAcquiredDx

,EncounterKey,DxName
466,D586C13398377F,*Unspecified


In [6]:
presentOnAdmitDx

,EncounterKey,DxName
671,D586C13398377F,*Unspecified


In [7]:
#change hospitalAcquiredDx to a json object
hospitalAcquiredDx = hospitalAcquiredDx.to_json(orient="records")
print(hospitalAcquiredDx)

[{"EncounterKey":"D586C13398377F","DxName":"*Unspecified"}]


In [8]:
#change presentOnAdmitDx to a json object
presentOnAdmitDx = presentOnAdmitDx.to_json(orient="records")
print(presentOnAdmitDx)

[{"EncounterKey":"D586C13398377F","DxName":"*Unspecified"}]


In [23]:
def extract_json_from_content(content):
    # Find the JSON part within the content
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_str = content[start_index:end_index]

    # Parse the JSON string
    parsed_json = json.loads(json_str)
    
    return parsed_json

In [ ]:
system = "You are a knowledgeable medical provider who specializes in medication management."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llama_3_1
response = chain.invoke({"text": f"""Based on the information from this JSON information: {hospitalAcquiredDx}, {presentOnAdmitDx}, does the patient have any of the following:
              1. Mild to moderate esoophagitis
              2. GERD 
              3. Peptic Ulcer Disease
              4. Upper GI symptoms
              5. ICU Stress Ulcer Prophylaxis
              6. Barrett's Esophagus
              7. Chronic NSAID use with bleeding risk
              8. Severe esophagitis
              9. Documented history of bleeding GI ulcer
              10. Explain the reasoning for your answer
            Return the answer for each of these as a formatted JSON object with the key being the condition and the value being a boolean value for the first 9.  For the final question, return a string with the reasoning for your answer."""})
print(response)

In [ ]:
extract_json_from_content(response.content)